In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Eczema/GSE120899'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A Phase 2 Randomized Trial of Apremilast in Patients With Atopic Dermatitis"
!Series_summary	"A phase 2, double-blind, placebo-controlled trial evaluated apremilast efficacy, safety, and pharmacodynamics in adults with moderate to severe atopic dermatitis (AD)."
!Series_overall_design	"Patients were randomized to placebo, apremilast 30 mg BID (APR30), or apremilast 40 mg BID (APR40) for 12 weeks. During Weeks 12–24, all patients received APR30 or APR40. A biopsy substudy evaluated AD-related biomarkers. Among 185 randomized intent-to-treat patients at Week 12, a dose-response relationship was observed; APR40 (n=63), but not APR30 (n=58), led to statistically significant improvements (vs. placebo [n=64]) in Eczema Area and Severity Index (mean [SD] percentage change from baseline: −31.6% [44.6] vs. −11.0% [71.2]; P<0.04; primary endpoint). mRNA expression of Th17/Th22-related markers (IL-17A, IL-22, S100A7/A8; P<0.05) showed the highest reductions 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine the availability of gene expression data
series_summary = "A phase 2, double-blind, placebo-controlled trial evaluated apremilast efficacy, safety, and pharmacodynamics in adults with moderate to severe atopic dermatitis (AD)."
if "mrna expression" in series_summary.lower():
    is_gene_available = True

# Data sample characteristics dictionary
sample_characteristics = {
    0: ['batch_date: 2016-02-01', 'batch_date: 2016-01-12', 'batch_date: 2016-01-20', 'batch_date: 2016-01-25'], 
    1: ['tissue: lesional skin', 'tissue: non-lesional skin', 'tissue: Normal'], 
    2: ['week: 0', 'week: 12', 'week: NA'], 
    3: ['treatment: APRMST-30', 'treatment: Placebo', 'treatment: APRMST-40', 'treatment: NA'], 
    4: ['patient id: 31007', 'patient id: 61001', 'patient id: 61007', 'patient id: 61013', 'patient id: 61015', 
        'patient id: 62012', 'patient id: 71001', 'patient id: 71004', 'patient id: 71005', 'patient id: 111002',
        'patient id: 111005', 'patient id: 2011004', 'patient id: 2011005', 'patient id: 2011006', 'patient id: 2011014', 
        'patient id: 2012017', 'patient id: 2021002', 'patient id: 3091001', 'patient id: 3091003', 'patient id: 3101001', 
        'patient id: 3101002', 'patient id: N5', 'patient id: N8']
}

# Determine the availability of the 'Eczema' trait
# No explicit 'Eczema' data found in sample_characteristics
trait_row = None

# There are no keys for age or gender in the current sample_characteristics
age_row = None
gender_row = None

# Define conversion functions as a formality, though they won't be used as per current dataset
def convert_trait(value):
    parts = value.split(': ')
    if len(parts) == 2:
        trait_value = parts[1].strip().lower()
        if trait_value == 'true':
            return 1
        elif trait_value == 'false':
            return 0
    return None
        
def convert_age(value):
    parts = value.split(': ')
    if len(parts) == 2:
        age_value = parts[1].strip()
        if age_value.isdigit():
            return int(age_value)
    return None

def convert_gender(value):
    parts = value.split(': ')
    if len(parts) == 2:
        gender_value = parts[1].strip().lower()
        if gender_value == 'female':
            return 0
        elif gender_value == 'male':
            return 1
    return None

# Save cohort information
save_cohort_info('GSE120899', './preprocessed/Eczema/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction (will be skipped due to None value of trait_row)
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Eczema', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Eczema/trait_data/GSE120899.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
